Teste

In [5]:
import requests
import pandas as pd

In [ ]:
url = "https://olinda.bcb.gov.br/olinda/servico/Informes_Agencias/versao/v1/odata/Agencias?$format=json"

response = requests.get(url)
data = response.json()

In [6]:
print(pd.DataFrame(data).columns)

# Extraia os dados da chave 'value'
agencias = data['value']

# Converta para DataFrame
df = pd.DataFrame(agencias)

# Exiba as primeiras linhas
print(df.head())
print(df.columns)

Index(['@odata.context', 'value'], dtype='object')
   CnpjBase CnpjSequencial CnpjDv                            NomeIf  \
0  07237373           0166     38  BANCO DO NORDESTE DO BRASIL S.A.   
1  00360305           4791     24           CAIXA ECONOMICA FEDERAL   
2  00000000           7687     23              BANCO DO BRASIL S.A.   
3  00000000           7686     42              BANCO DO BRASIL S.A.   
4  90400888           1970     00     BANCO SANTANDER (BRASIL) S.A.   

                  Segmento CodigoCompe                 NomeAgencia  \
0           Banco Múltiplo       00166                  ACAILANDIA   
1  Caixa Econômica Federal       04791                 ITARIRI, SP   
2           Banco Múltiplo       09190        ESCR.EXC.RUDGE RAMOS   
3           Banco Múltiplo       09189          ESCR.EXC.S.B.CAMPO   
4           Banco Múltiplo       04738  001 4738 TAGUATINGA -DF II   

                         Endereco Numero    Complemento        Bairro  \
0  AV. DORGIVAL PINHEIRO DE 

In [11]:
import os
import requests
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

In [15]:
api = "https://olinda.bcb.gov.br/olinda/servico/Informes_Agencias/versao/v1/odata/Agencias?$format=json"
resposta = requests.get(api)
data = resposta.json()
data

{'@odata.context': 'https://was-p.bcnet.bcb.gov.br/olinda/servico/Informes_Agencias/versao/v1/odata$metadata#Agencias',
 'value': [{'CnpjBase': '07237373',
   'CnpjSequencial': '0217',
   'CnpjDv': '12',
   'NomeIf': 'BANCO DO NORDESTE DO BRASIL S.A.',
   'Segmento': 'Banco Múltiplo',
   'CodigoCompe': '00217',
   'NomeAgencia': 'ALMENARA',
   'Endereco': 'PRAÇA DR HÉLIO ROCHA GUIMARÃES',
   'Numero': '',
   'Complemento': '',
   'Bairro': 'CENTRO',
   'Cep': '39900-000',
   'MunicipioIbge': '3101706',
   'Municipio': 'ALMENARA',
   'UF': 'MG',
   'DataInicio': '26/10/2011',
   'DDD': '33',
   'Telefone': '37219600',
   'Posicao': '10/06/2025'},
  {'CnpjBase': '00360305',
   'CnpjSequencial': '4791',
   'CnpjDv': '24',
   'NomeIf': 'CAIXA ECONOMICA FEDERAL',
   'Segmento': 'Caixa Econômica Federal',
   'CodigoCompe': '04791',
   'NomeAgencia': 'ITARIRI, SP',
   'Endereco': 'AVENIDA JOSE FERREIRA FRANCO',
   'Numero': '',
   'Complemento': '',
   'Bairro': 'CENTRO',
   'Cep': '11760-000

In [13]:
agencias = data.get("value", [])

# cria um documento em lagnchain
documents = []
i = 0

for agencia in agencias:
    content = ', '.join(f"{chave}: {valor}" for chave, valor in agencia.items())
    metadata = {
        "contexto": "Dados de agências de instituições supervisionadas pelo Banco Central do Brasil (Bacen)"
    }
    documents.append(Document(page_content=content, metadata=metadata))
    i+=1
    if i == 10: break

modelo = OllamaEmbeddings(model="nomic-embed-text")
persist_directory = os.path.join(os.getcwd(), "teste")

# cria um vetor de pesos a partir do documento passado
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=modelo,
    persist_directory=persist_directory
)
vectorstore.persist()

print(f"Vetorstore salvo em: {persist_directory}")

Vetorstore salvo em: c:\Users\Plusb\Documents\ic\teste


C:\Users\Plusb\AppData\Local\Temp\ipykernel_11432\679395021.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [9]:
for agencia in agencias:
    df = pd.DataFrame([agencia])
    print(df.head())
    break

   CnpjBase CnpjSequencial CnpjDv                            NomeIf  \
0  07237373           0166     38  BANCO DO NORDESTE DO BRASIL S.A.   

         Segmento CodigoCompe NomeAgencia                        Endereco  \
0  Banco Múltiplo       00166  ACAILANDIA  AV. DORGIVAL PINHEIRO DE SOUZA   

  Numero Complemento  Bairro        Cep MunicipioIbge   Municipio  UF  \
0                     CENTRO  65930-000       2100055  ACAILANDIA  MA   

   DataInicio DDD  Telefone     Posicao  
0  10/01/1985  99  35381180  03/06/2025  


Embeeding

Bibliotecas

In [4]:
import os
import time
import requests
import json
import signal
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

Configurações

In [ ]:
# BASE_DIR aponta para a pasta que contém cdg e dados
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))

# Diretórios de destino dentro da pasta dados
DADOS_DIR = os.path.join(BASE_DIR, "dados")
BRUTO_DIR = os.path.join(DADOS_DIR, "bruto")
CHECKPOINT_DIR = os.path.join(DADOS_DIR, "checkpoints")
FAISS_DIR = os.path.join(DADOS_DIR, "dados_faiss")

# Cria os diretórios se ainda não existirem
os.makedirs(BRUTO_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(FAISS_DIR, exist_ok=True)

# Caminhos completos para os arquivos
JSONL_FILE = os.path.join(BRUTO_DIR, "agencias.jsonl")
CHECKPOINT_FILE = os.path.join(CHECKPOINT_DIR, "checkpoint.txt")

# URL da API (inalterada)
api = "https://olinda.bcb.gov.br/olinda/servico/Informes_Agencias/versao/v1/odata/Agencias?$format=json"


Código

In [8]:
# Flag global para controle da interrupção via Ctrl+C
PARE = False

def handler(sig, frame):
    """
    Handler do sinal SIGINT (Ctrl+C).
    Quando o usuário interrompe a execução, seta a flag PARE para True,
    permitindo finalizar o batch atual antes de parar o programa.
    """
    global PARE
    print("\nCtrl+C detectado. Finalizando o batch atual antes de parar...")
    PARE = True

# Associa o handler ao sinal SIGINT para interrupção segura
signal.signal(signal.SIGINT, handler)

def download_jsonl():
    """
    Baixa os dados da API e salva em um arquivo JSONL.
    Retorna a quantidade de registros baixados.
    """
    print("Baixando dados da API")
    resposta = requests.get(api)
    data = resposta.json()
    dados = data.get("value", [])  # Pega a lista de dados da chave 'value', específica da API Bacen

    # Abre o arquivo JSONL para escrita, codificação UTF-8
    with open(JSONL_FILE, "w", encoding="utf-8") as f:
        for dado in dados:
            json_line = json.dumps(dado, ensure_ascii=False)  # Converte cada registro para JSON string
            f.write(json_line + "\n")  # Escreve uma linha por registro
            
    print(f"Dados brutos salvos em: {JSONL_FILE}")
    return len(dados)

def load_checkpoint():
    """
    Lê o arquivo de checkpoint que armazena a linha onde o processamento parou.
    Retorna a linha para continuar o processamento ou 0 se não existir.
    """
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            linha = f.read().strip()
            return int(linha) if linha.isdigit() else 0
    return 0

def save_checkpoint(linha):
    """
    Salva a próxima linha a ser processada no arquivo de checkpoint.
    Isso permite retomar o processamento em caso de interrupção.
    """
    with open(CHECKPOINT_FILE, "w") as f:
        f.write(str(linha))

def process_embedding(batch_size=10):
    """
    Processa os dados JSONL em batches para gerar embeddings,
    salvando-os em um índice FAISS local para recuperação eficiente.
    
    O processamento pode ser interrompido com Ctrl+C, mas o batch atual será finalizado antes.
    """
    
    modelo = OllamaEmbeddings(model="nomic-embed-text")  # Modelo de embeddings usado
    linha_inicial = load_checkpoint()  # Retoma do checkpoint
    documents = []  # Lista temporária para agrupar documentos no batch
    total_linhas = sum(1 for _ in open(JSONL_FILE, "r", encoding="utf-8"))  # Total de linhas no arquivo

    # Verifica se existe índice FAISS salvo para atualizar
    if os.listdir(FAISS_DIR):
        vectorstore = FAISS.load_local(FAISS_DIR, embeddings=modelo, allow_dangerous_deserialization=True)
        # 'allow_dangerous_deserialization=True' porque o arquivo é gerado localmente e confiável
        print("Índice FAISS carregado para atualização.")
    else:
        vectorstore = None
        print("Nenhum índice FAISS encontrado, criando um novo.")

    # Abre o arquivo JSONL para leitura
    with open(JSONL_FILE, "r", encoding="utf-8") as f:
        
        i = linha_inicial - 1  # inicializa para o caso de não entrar no loop
        
        for i, linha in enumerate(f):
            if i < linha_inicial:
                # Pula linhas já processadas segundo o checkpoint
                continue

            ti = time.time()  # Marca início do processamento do batch

            data = json.loads(linha)  # Converte a linha JSON em dicionário
            # Concatena os campos e valores para o conteúdo do documento
            content = ', '.join(f"{chave}: {valor}" for chave, valor in data.items())
            metadata = {
                "contexto": "Dados de agências de instituições supervisionadas pelo Banco Central do Brasil (Bacen)"
            }
            documents.append(Document(page_content=content, metadata=metadata))  # Cria documento para o batch

            # Só processa o batch quando atingir o tamanho definido
            if len(documents) >= batch_size:

                if vectorstore is None:
                    # Cria novo índice FAISS se não existir
                    vectorstore = FAISS.from_documents(documents, embedding=modelo)
                else:
                    # Adiciona documentos ao índice existente
                    vectorstore.add_documents(documents)

                vectorstore.save_local(FAISS_DIR)  # Salva índice atualizado localmente

                tt = time.time() - ti  # Calcula tempo gasto no batch
                print(f"Processado lote até linha {i+1} / {total_linhas} - Tempo: {tt:.2f} s")

                save_checkpoint(i + 1)  # Salva checkpoint para próxima linha
                documents.clear()  # Limpa documentos do batch para o próximo ciclo

                # Se o usuário pediu para parar, encerra após finalizar o batch
                if PARE:
                    print("Interrupção segura concluída após completar o batch. Encerrando processo.")
                    break
                
        # Após o for, fora do loop
        if documents:
            # Processa os documentos restantes mesmo que não completem o batch
            if vectorstore is None:
                vectorstore = FAISS.from_documents(documents, embedding=modelo)
            else:
                vectorstore.add_documents(documents)

            vectorstore.save_local(FAISS_DIR)
            print(f"Processado último lote incompleto com {len(documents)} documentos.")
            save_checkpoint(i + 1)  # salva checkpoint final

    # Se terminou sem interrupção
    if not PARE:
        print("Processamento finalizado normalmente.")
        print(f"Índice salvo em: {FAISS_DIR}")

# Execução principal
if not os.path.exists(JSONL_FILE):  # Se os dados ainda não foram baixados
    total = download_jsonl()
    print(f"Total de registros baixados: {total}")
else:
    print("Arquivo JSONL já existe, pulando download.")

process_embedding(batch_size=10)  # Processa embeddings em pacotes de 10

Arquivo JSONL já existe, pulando download.
Índice FAISS carregado para atualização.
Processado lote até linha 70 / 16249 - Tempo: 22.43 s

Ctrl+C detectado. Finalizando o batch atual antes de parar...
Processado lote até linha 80 / 16249 - Tempo: 22.44 s
Interrupção segura concluída após completar o batch. Encerrando processo.
